In [1]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import csv
import os
import pickle
from pathlib import Path
from tqdm import tqdm
from PIL import ImageOps, Image

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose

# Push-up

## Get images path
Only pushup-up class

In [9]:
image_dirs = ['/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out']
accepted_exts = ('.jpg', '.jpeg','.png')
all_file_paths = []
for image_dir in image_dirs:
    for path, subdir, files in os.walk(image_dir):
        for file in files:
            if file.lower().endswith(accepted_exts):
                full_path = os.path.join(path, file)
                print(full_path)
                all_file_paths.append(full_path)

/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/64_2_1.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/16_4_1.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/13_2_1.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/86_2_1.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/68_1_2.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/101_1_2.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/100_2_1.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/97_2_1.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/31_3_2.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/36_2_1.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/103_1_2.jpg
/home/u6188041/jom/Dataset/EzFit_dataset

In [10]:
len(all_file_paths)

252

## Extract keypoints from path

In [11]:
SEQ_LEN = 8
data = []
error_files = []
for image_path in tqdm(all_file_paths):
    image = cv2.imread(image_path)
    with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
        with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose2:
            results = pose.process(image)

            try:
                pose_coor = results.pose_landmarks.landmark
                # no need z here
                body_kps = np.array([[landmark.x, landmark.y, landmark.visibility] for landmark in pose_coor]).flatten()
                stacked_kps = np.tile(body_kps, (SEQ_LEN, 1))
                data.append(stacked_kps)

            except:
                error_files.append(image_path)
                
            # augment flip horizontal
            flipped_image = np.flip(image, axis=1)   
            aug_results = pose2.process(flipped_image)
            
            try:
                pose_coor = aug_results.pose_landmarks.landmark
                # no need z here
                body_kps = np.array([[landmark.x, landmark.y, landmark.visibility] for landmark in pose_coor]).flatten()
                stacked_kps = np.tile(body_kps, (SEQ_LEN, 1))
                data.append(stacked_kps)

            except:
                error_files.append('aug_'+image_path)

  0%|                                                                                           | 0/252 [00:00<?, ?it/s]INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.
  0%|▎                                                                                  | 1/252 [00:00<00:58,  4.30it/s]INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.
INFO: Replacing 318 node(s) with delega

In [12]:
error_files

['/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/31_3_2.jpg',
 'aug_/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/31_3_2.jpg',
 'aug_/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/72_1_2.jpg',
 '/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/47_2_1.jpg',
 'aug_/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/47_2_1.jpg',
 'aug_/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/21_1_2.jpg',
 '/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/49_1_2.jpg',
 'aug_/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/49_1_2.jpg',
 '/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/94_2_1.jpg',
 'aug_/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/half-burpee-out/94_2_1.jpg',
 'aug_/home/u6188041/jom/Dataset/EzFit_dataset/images/half-burpee/

In [13]:
data = np.array(data)
data.shape

(474, 8, 99)

In [14]:
1500/474

3.1645569620253164

In [15]:
# stack data more
STACK_MORE = 3
stacked_data = np.tile(data, (STACK_MORE, 1))
stacked_data.shape

(474, 24, 99)

In [16]:
stacked_data = stacked_data.reshape((-1, 8, 99))
stacked_data

array([[[0.51717603, 0.64744323, 0.99999726, ..., 0.64602202,
         0.8099426 , 0.87483799],
        [0.51717603, 0.64744323, 0.99999726, ..., 0.64602202,
         0.8099426 , 0.87483799],
        [0.51717603, 0.64744323, 0.99999726, ..., 0.64602202,
         0.8099426 , 0.87483799],
        ...,
        [0.51717603, 0.64744323, 0.99999726, ..., 0.64602202,
         0.8099426 , 0.87483799],
        [0.51717603, 0.64744323, 0.99999726, ..., 0.64602202,
         0.8099426 , 0.87483799],
        [0.51717603, 0.64744323, 0.99999726, ..., 0.64602202,
         0.8099426 , 0.87483799]],

       [[0.51717603, 0.64744323, 0.99999726, ..., 0.64602202,
         0.8099426 , 0.87483799],
        [0.51717603, 0.64744323, 0.99999726, ..., 0.64602202,
         0.8099426 , 0.87483799],
        [0.51717603, 0.64744323, 0.99999726, ..., 0.64602202,
         0.8099426 , 0.87483799],
        ...,
        [0.51717603, 0.64744323, 0.99999726, ..., 0.64602202,
         0.8099426 , 0.87483799],
        [0.5

In [17]:
stacked_data.shape

(1422, 8, 99)

In [19]:
np.min(stacked_data), np.mean(stacked_data),np.max(stacked_data)

(-0.06879878044128418, 0.6811412141728583, 1.030326008796692)

In [20]:
np.random.seed(42)
noise = np.random.normal(scale=0.01, size=stacked_data.shape[:3])
noise.shape

(1422, 8, 99)

In [21]:
noise.min(), noise.mean(), noise.max()

(-0.04829436008511157, -1.3203377271758534e-05, 0.04678949100376724)

In [15]:
noisy_stacked_data = stacked_data + noise

In [16]:
noisy_stacked_data.min(), noisy_stacked_data.max()

(-0.08907562320719736, 1.0552049585009864)

In [17]:
np.save('saved_data/half-burpee_idle_class_len_8', noisy_stacked_data)

# Save to .npy files

In [18]:
idle_data = np.load('saved_data/half-burpee_idle_class_len_8.npy')
idle_data.shape

(1422, 8, 99)

In [19]:
idle_data[0,:].shape

(8, 99)

In [20]:
save_dir = Path('D:/EzFit_dataset/keypoints_seq/len_8/half-burpee/idle')
for idx in tqdm(range(idle_data.shape[0])):
    base_dir = save_dir / str(idx)
    if not base_dir.exists():
        base_dir.mkdir(parents=True)
    selected = idle_data[idx,:]
    np.save(base_dir / f'idle_{idx}', selected)

100%|█████████████████████████████████████████████████████████████████████████████| 1422/1422 [00:03<00:00, 440.29it/s]
